# DATA CLEANING


In [ ]:
import pandas as pd
import numpy as np

# Read the CSV file to get the 'Total' column name from the second row
df_header = pd.read_csv('GovtBonds+FPIs.csv', header=1, nrows=0)  # header=1 means the second row (0-indexed)

# Read the CSV file, specifying the header row for data
df = pd.read_csv('GovtBonds+FPIs.csv', header=2, parse_dates=['Date'])  # header=2 means the third row (0-indexed)

# Manually set the 'Total' column name
df.columns = list(df.columns[:2]) + ['Total'] + list(df.columns[3:])

# Create a DataFrame with Date, YLD_YTM_MID, and Total columns
yield_total_df = df[['Date', 'YLD_YTM_MID', 'Total']].copy()

# Display the DataFrame
print(yield_total_df)

KeyError: "['YLD_YTM_MID'] not in index"